In [4]:
# ============================================================================
# MODUL PRAKTIKUM PYSPARK - DATA PREPROCESSING
# Bagian: SETUP + LATIHAN + TUGAS LENGKAP
# ============================================================================

# ----------------------------------------------------------------------------
# LANGKAH 0: SETUP ENVIRONMENT
# ----------------------------------------------------------------------------
print("=" * 70)
print("LANGKAH 0: INSTALASI DAN SETUP")
print("=" * 70)

# Install PySpark (jalankan ini di Google Colab)
!pip install pyspark findspark

# Import libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler
from pyspark.ml.feature import Bucketizer, StringIndexer, OneHotEncoder
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

# Buat SparkSession (ini kayak "nyalain mesin Spark")
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PraktikumPreprocessing") \
    .getOrCreate()

print("✅ SparkSession berhasil dibuat!")
print()
# Note: The PySparkTypeError reported in cell jraVvRHR4my8 is due to an implicit call
# to PySpark's round function with a float argument. The fix was applied by using
# Python's built-in round() function on 'mean_rating' before imputation.
# If the error persists, please ensure cell jraVvRHR4my8 has been re-executed.

LANGKAH 0: INSTALASI DAN SETUP
✅ SparkSession berhasil dibuat!



In [5]:
print("=" * 70)
print("LANGKAH 1: MEMBUAT DATASET SAMPEL")
print("=" * 70)

# Data sampel yang "kotor" (sengaja ada error buat latihan)
data_kotor = [
    (1, 'Budi Susanto', 25, 5500000, 'L', '2022-01-15', 'Jakarta', 'Transaksi berhasil, barang bagus'),
    (2, 'Ani Lestari', None, 8000000, 'P', '2022-02-20', 'Bandung', 'Pengiriman cepat dan barang sesuai'),
    (3, 'Candra Wijaya', 35, 12000000, 'L', '2022-01-18', 'Surabaya', 'Sangat puas dengan pelayanannya'),
    (4, 'Dewi Anggraini', 22, 4800000, 'P', '2022-03-10', 'JKT', 'Barang diterima dalam kondisi baik'),
    (5, 'Eka Prasetyo', 45, 15000000, 'L', '2022-04-01', 'Jakarta', 'Transaksi gagal, mohon diperiksa'),
    (6, 'Budi Susanto', 25, 5500000, 'L', '2022-01-15', 'Jakarta', 'Transaksi berhasil, barang bagus'),  # Duplikat
    (7, 'Fina Rahmawati', 29, 9500000, 'P', '2022-05-12', 'Bandung', None),  # Missing value
    (8, 'Galih Nugroho', 31, -7500000, 'L', '2022-06-25', 'Surabaya', 'Barang oke'),  # Gaji negatif (noise)
    (9, 'Hesti Wulandari', 55, 25000000, 'P', '2022-07-30', 'Jakarta', 'Pelayanan ramah dan cepat'),
    (10, 'Indra Maulana', 150, 6200000, 'L', '2022-08-05', 'Medan', 'Produk original')  # Usia 150 (outlier)
]

# Definisikan skema (struktur kolom DataFrame)
skema = StructType([
    StructField("id_pelanggan", IntegerType(), True),
    StructField("nama", StringType(), True),
    StructField("usia", IntegerType(), True),
    StructField("gaji", IntegerType(), True),
    StructField("jenis_kelamin", StringType(), True),
    StructField("tgl_registrasi", StringType(), True),
    StructField("kota", StringType(), True),
    StructField("ulasan", StringType(), True)
])

# Buat DataFrame dari data kotor
df = spark.createDataFrame(data=data_kotor, schema=skema)

print("Dataset Awal (Kotor):")
df.show(truncate=False)
print()

LANGKAH 1: MEMBUAT DATASET SAMPEL
Dataset Awal (Kotor):
+------------+---------------+----+--------+-------------+--------------+--------+----------------------------------+
|id_pelanggan|nama           |usia|gaji    |jenis_kelamin|tgl_registrasi|kota    |ulasan                            |
+------------+---------------+----+--------+-------------+--------------+--------+----------------------------------+
|1           |Budi Susanto   |25  |5500000 |L            |2022-01-15    |Jakarta |Transaksi berhasil, barang bagus  |
|2           |Ani Lestari    |NULL|8000000 |P            |2022-02-20    |Bandung |Pengiriman cepat dan barang sesuai|
|3           |Candra Wijaya  |35  |12000000|L            |2022-01-18    |Surabaya|Sangat puas dengan pelayanannya   |
|4           |Dewi Anggraini |22  |4800000 |P            |2022-03-10    |JKT     |Barang diterima dalam kondisi baik|
|5           |Eka Prasetyo   |45  |15000000|L            |2022-04-01    |Jakarta |Transaksi gagal, mohon diperiksa  |


In [6]:
# ----------------------------------------------------------------------------
# BAGIAN 1: DATA CLEANING
# ----------------------------------------------------------------------------
print("=" * 70)
print("BAGIAN 1: DATA CLEANING")
print("=" * 70)

# 1.1 Cek Missing Values
print("\n1.1 Missing Values per Kolom:")
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

# 1.2 Imputasi (Isi nilai kosong)
mean_usia = df.select(mean(df['usia'])).collect()[0][0]
df_filled = df.na.fill({'usia': int(mean_usia)})
df_filled = df_filled.na.fill({'ulasan': 'Tidak ada ulasan'})
print("Data setelah imputasi:")
df_filled.show(truncate=False)

# 1.3 Hapus Duplikat
print(f"\nJumlah baris sebelum hapus duplikat: {df_filled.count()}")
df_bersih = df_filled.dropDuplicates()
print(f"Jumlah baris setelah hapus duplikat: {df_bersih.count()}")

# 1.4 Perbaiki Data Inkonsisten & Noisy
df_bersih = df_bersih.withColumn("kota",
    when(df_bersih["kota"] == "JKT", "Jakarta").otherwise(df_bersih["kota"]))
df_bersih = df_bersih.withColumn("gaji", abs(df_bersih["gaji"]))
df_bersih = df_bersih.filter(df_bersih["usia"] <= 100)

print("\nData setelah cleaning:")
df_bersih.show()
print()

BAGIAN 1: DATA CLEANING

1.1 Missing Values per Kolom:
+------------+----+----+----+-------------+--------------+----+------+
|id_pelanggan|nama|usia|gaji|jenis_kelamin|tgl_registrasi|kota|ulasan|
+------------+----+----+----+-------------+--------------+----+------+
|           0|   0|   1|   0|            0|             0|   0|     1|
+------------+----+----+----+-------------+--------------+----+------+

Data setelah imputasi:
+------------+---------------+----+--------+-------------+--------------+--------+----------------------------------+
|id_pelanggan|nama           |usia|gaji    |jenis_kelamin|tgl_registrasi|kota    |ulasan                            |
+------------+---------------+----+--------+-------------+--------------+--------+----------------------------------+
|1           |Budi Susanto   |25  |5500000 |L            |2022-01-15    |Jakarta |Transaksi berhasil, barang bagus  |
|2           |Ani Lestari    |46  |8000000 |P            |2022-02-20    |Bandung |Pengiriman c

In [7]:
# ============================================================================
# BAGIAN 4.1: LATIHAN
# ============================================================================
print("=" * 70)
print("BAGIAN 4.1: LATIHAN")
print("=" * 70)

# ----------------------------------------------------------------------------
# LATIHAN 1: Agregasi Lanjutan
# ----------------------------------------------------------------------------
print("\n📝 LATIHAN 1: Agregasi berdasarkan jenis_kelamin dan kota")
print("-" * 70)

df_agregasi_latihan = df_bersih.groupBy("jenis_kelamin", "kota") \
    .agg(
        max("gaji").alias("gaji_maksimum"),
        min("usia").alias("usia_minimum")
    ) \
    .orderBy("jenis_kelamin", "kota")

print("Hasil Agregasi:")
df_agregasi_latihan.show()

# PENJELASAN:
# - groupBy(): kelompokkan data berdasarkan jenis_kelamin DAN kota
# - max("gaji"): cari gaji tertinggi di tiap grup
# - min("usia"): cari usia terendah di tiap grup
# - orderBy(): urutkan hasil supaya rapi

BAGIAN 4.1: LATIHAN

📝 LATIHAN 1: Agregasi berdasarkan jenis_kelamin dan kota
----------------------------------------------------------------------
Hasil Agregasi:
+-------------+--------+-------------+------------+
|jenis_kelamin|    kota|gaji_maksimum|usia_minimum|
+-------------+--------+-------------+------------+
|            L| Jakarta|     15000000|          25|
|            L|Surabaya|     12000000|          31|
|            P| Bandung|      9500000|          29|
|            P| Jakarta|     25000000|          22|
+-------------+--------+-------------+------------+



In [8]:
# ----------------------------------------------------------------------------
# LATIHAN 2: Diskretisasi Gaji
# ----------------------------------------------------------------------------
print("\n📝 LATIHAN 2: Diskretisasi Gaji menjadi Level")
print("-" * 70)

# Definisikan batasan untuk binning
# [0, 7jt, 15jt, infinity]
splits_gaji = [0, 7000000, 15000000, float('Inf')]

bucketizer_gaji = Bucketizer(
    splits=splits_gaji,
    inputCol="gaji",
    outputCol="level_gaji_index"
)

df_gaji_binned = bucketizer_gaji.transform(df_bersih)

# Ubah index menjadi label yang readable
df_gaji_binned = df_gaji_binned.withColumn("level_gaji",
    when(col("level_gaji_index") == 0.0, "Rendah")
    .when(col("level_gaji_index") == 1.0, "Menengah")
    .when(col("level_gaji_index") == 2.0, "Tinggi")
    .otherwise("Unknown")
)

print("Data dengan Level Gaji:")
df_gaji_binned.select("id_pelanggan", "nama", "gaji", "level_gaji").show()

# PENJELASAN:
# - Bucketizer: membagi data kontinu (gaji) ke dalam "bucket"/kategori
# - splits: batasan untuk tiap kategori
# - withColumn + when: buat kolom baru dengan kondisi if-else


📝 LATIHAN 2: Diskretisasi Gaji menjadi Level
----------------------------------------------------------------------
Data dengan Level Gaji:
+------------+---------------+--------+----------+
|id_pelanggan|           nama|    gaji|level_gaji|
+------------+---------------+--------+----------+
|           5|   Eka Prasetyo|15000000|    Tinggi|
|           2|    Ani Lestari| 8000000|  Menengah|
|           1|   Budi Susanto| 5500000|    Rendah|
|           3|  Candra Wijaya|12000000|  Menengah|
|           4| Dewi Anggraini| 4800000|    Rendah|
|           8|  Galih Nugroho| 7500000|  Menengah|
|           9|Hesti Wulandari|25000000|    Tinggi|
|           7| Fina Rahmawati| 9500000|  Menengah|
|           6|   Budi Susanto| 5500000|    Rendah|
+------------+---------------+--------+----------+



In [9]:
# ----------------------------------------------------------------------------
# LATIHAN 3: Feature Engineering Sederhana (Interaksi)
# ----------------------------------------------------------------------------
print("\n📝 LATIHAN 3: Feature Engineering - Interaksi usia_x_gaji")
print("-" * 70)

df_interaksi = df_bersih.withColumn("usia_x_gaji", col("usia") * col("gaji"))

print("5 Baris Pertama dengan Feature Interaksi:")
df_interaksi.select("id_pelanggan", "usia", "gaji", "usia_x_gaji").show(5)

# PENJELASAN:
# - Feature interaksi: kombinasi 2+ fitur untuk capture hubungan kompleks
# - usia_x_gaji bisa berguna untuk model ML (misal: prediksi daya beli)

print("\n✅ LATIHAN 1-3 SELESAI!\n")


📝 LATIHAN 3: Feature Engineering - Interaksi usia_x_gaji
----------------------------------------------------------------------
5 Baris Pertama dengan Feature Interaksi:
+------------+----+--------+-----------+
|id_pelanggan|usia|    gaji|usia_x_gaji|
+------------+----+--------+-----------+
|           5|  45|15000000|  675000000|
|           2|  46| 8000000|  368000000|
|           1|  25| 5500000|  137500000|
|           3|  35|12000000|  420000000|
|           4|  22| 4800000|  105600000|
+------------+----+--------+-----------+
only showing top 5 rows

✅ LATIHAN 1-3 SELESAI!



In [10]:
# ============================================================================
# BAGIAN 4.2: TUGAS
# ============================================================================
print("=" * 70)
print("BAGIAN 4.2: TUGAS - Data Produk Elektronik")
print("=" * 70)

# ----------------------------------------------------------------------------
# Buat Dataset Tugas
# ----------------------------------------------------------------------------
data_produk = [
    (101, 'Laptop A', 'Elektronik', 15000000, 4.5, 120, '2023-01-20', 'stok_tersedia'),
    (102, 'Smartphone B', 'Elektronik', 8000000, 4.7, 250, '2023-02-10', 'stok_tersedia'),
    (103, 'Headphone C', 'Aksesoris', 1200000, 4.2, None, '2023-02-15', 'stok_habis'),
    (104, 'Laptop A', 'Elektronik', 15000000, 4.5, 120, '2023-01-20', 'stok_tersedia'),  # Duplikat
    (105, 'Tablet D', 'Elektronik', 6500000, None, 80, '2023-03-01', 'stok_tersedia'),
    (106, 'Charger E', 'Aksesoris', 250000, -4.0, 500, '2023-03-05', 'Stok_Tersedia'),  # Rating negatif, status inkonsisten
    (107, 'Smartwatch F', 'Elektronik', 3100000, 4.8, 150, '2023-04-12', 'stok_habis')
]

skema_produk = StructType([
    StructField("id_produk", IntegerType()),
    StructField("nama_produk", StringType()),
    StructField("kategori", StringType()),
    StructField("harga", IntegerType()),
    StructField("rating", FloatType()),
    StructField("terjual", IntegerType()),
    StructField("tgl_rilis", StringType()),
    StructField("status_stok", StringType())
])

df_tugas = spark.createDataFrame(data=data_produk, schema=skema_produk)

print("\n📦 Dataset Produk Elektronik (Awal - Kotor):")
df_tugas.show()


BAGIAN 4.2: TUGAS - Data Produk Elektronik

📦 Dataset Produk Elektronik (Awal - Kotor):
+---------+------------+----------+--------+------+-------+----------+-------------+
|id_produk| nama_produk|  kategori|   harga|rating|terjual| tgl_rilis|  status_stok|
+---------+------------+----------+--------+------+-------+----------+-------------+
|      101|    Laptop A|Elektronik|15000000|   4.5|    120|2023-01-20|stok_tersedia|
|      102|Smartphone B|Elektronik| 8000000|   4.7|    250|2023-02-10|stok_tersedia|
|      103| Headphone C| Aksesoris| 1200000|   4.2|   NULL|2023-02-15|   stok_habis|
|      104|    Laptop A|Elektronik|15000000|   4.5|    120|2023-01-20|stok_tersedia|
|      105|    Tablet D|Elektronik| 6500000|  NULL|     80|2023-03-01|stok_tersedia|
|      106|   Charger E| Aksesoris|  250000|  -4.0|    500|2023-03-05|Stok_Tersedia|
|      107|Smartwatch F|Elektronik| 3100000|   4.8|    150|2023-04-12|   stok_habis|
+---------+------------+----------+--------+------+-------+---

In [11]:
# ----------------------------------------------------------------------------
# TUGAS 1: DATA CLEANING
# ----------------------------------------------------------------------------
print("\n" + "=" * 70)
print("TUGAS 1: DATA CLEANING")
print("=" * 70)

# 1a. Tangani Missing Values
print("\n🔍 Missing Values per Kolom:")
df_tugas.select([count(when(isnull(c), c)).alias(c) for c in df_tugas.columns]).show()

# Imputasi:
# - terjual: isi dengan MEDIAN (lebih robust untuk data penjualan)
# - rating: isi dengan MEAN (karena rating biasanya terdistribusi normal)

# Hitung median terjual (PySpark ga ada median langsung, pakai percentile)
median_terjual = df_tugas.approxQuantile("terjual", [0.5], 0.01)[0]
mean_rating = df_tugas.select(mean("rating")).collect()[0][0]

# Pre-round mean_rating using Python's built-in round to avoid PySpark's round function conflict
rounded_mean_rating = __builtins__.round(mean_rating, 1)

df_tugas_clean = df_tugas.na.fill({
    'terjual': int(median_terjual),
    'rating': rounded_mean_rating
})

print(f"\n✅ Imputasi: terjual={int(median_terjual)}, rating={rounded_mean_rating}")
df_tugas_clean.show()

# 1b. Hapus Duplikat
print(f"\nJumlah baris sebelum hapus duplikat: {df_tugas_clean.count()}")
df_tugas_clean = df_tugas_clean.dropDuplicates()
print(f"Jumlah baris setelah hapus duplikat: {df_tugas_clean.count()}")

# 1c. Perbaiki Rating Negatif (ambil nilai absolut)
df_tugas_clean = df_tugas_clean.withColumn("rating", abs(col("rating")))

# 1d. Standarisasi status_stok (lowercase semua)
df_tugas_clean = df_tugas_clean.withColumn("status_stok", lower(col("status_stok")))

print("\n✅ Data setelah Cleaning:")
df_tugas_clean.show()



TUGAS 1: DATA CLEANING

🔍 Missing Values per Kolom:
+---------+-----------+--------+-----+------+-------+---------+-----------+
|id_produk|nama_produk|kategori|harga|rating|terjual|tgl_rilis|status_stok|
+---------+-----------+--------+-----+------+-------+---------+-----------+
|        0|          0|       0|    0|     1|      1|        0|          0|
+---------+-----------+--------+-----+------+-------+---------+-----------+


✅ Imputasi: terjual=120, rating=3.1
+---------+------------+----------+--------+------+-------+----------+-------------+
|id_produk| nama_produk|  kategori|   harga|rating|terjual| tgl_rilis|  status_stok|
+---------+------------+----------+--------+------+-------+----------+-------------+
|      101|    Laptop A|Elektronik|15000000|   4.5|    120|2023-01-20|stok_tersedia|
|      102|Smartphone B|Elektronik| 8000000|   4.7|    250|2023-02-10|stok_tersedia|
|      103| Headphone C| Aksesoris| 1200000|   4.2|    120|2023-02-15|   stok_habis|
|      104|    Lapt

In [12]:
# ----------------------------------------------------------------------------
# TUGAS 2: DATA TRANSFORMATION (Standarisasi)
# ----------------------------------------------------------------------------
print("\n" + "=" * 70)
print("TUGAS 2: DATA TRANSFORMATION - Standarisasi")
print("=" * 70)

# Gabungkan kolom numerik ke dalam 1 vektor
assembler_tugas = VectorAssembler(
    inputCols=["harga", "rating", "terjual"],
    outputCol="fitur_numerik"
)
df_tugas_vector = assembler_tugas.transform(df_tugas_clean)

# Standarisasi (Z-score normalization)
scaler_tugas = StandardScaler(
    inputCol="fitur_numerik",
    outputCol="fitur_standar",
    withStd=True,
    withMean=True
)
scaler_model_tugas = scaler_tugas.fit(df_tugas_vector)
df_tugas_scaled = scaler_model_tugas.transform(df_tugas_vector)

print("\n✅ Data setelah Standarisasi:")
df_tugas_scaled.select("nama_produk", "harga", "rating", "terjual", "fitur_standar").show(truncate=False)

# PENJELASAN STANDARISASI:
# - Ubah nilai jadi mean=0, std=1
# - Berguna supaya semua fitur punya skala yang sama
# - Contoh: harga (jutaan) vs rating (0-5) → jadi sebanding


TUGAS 2: DATA TRANSFORMATION - Standarisasi

✅ Data setelah Standarisasi:
+------------+--------+------+-------+--------------------------------------------------------------+
|nama_produk |harga   |rating|terjual|fitur_standar                                                 |
+------------+--------+------+-------+--------------------------------------------------------------+
|Headphone C |1200000 |4.2   |120    |[-0.9511344112633691,-0.09855311527358197,-0.4888743395612589]|
|Laptop A    |15000000|4.5   |120    |[1.3091259608901722,0.4188497119263139,-0.4888743395612589]   |
|Smartphone B|8000000 |4.7   |250    |[0.16261707646446283,0.7637843818001624,0.40087695844023213]  |
|Smartwatch F|3100000 |4.8   |150    |[-0.6399391426335337,0.9362525391262098,-0.28354711694553025] |
|Tablet D    |6500000 |3.1   |80     |[-0.08306339876961774,-1.995695444357995,-0.7626439697155639] |
|Charger E   |250000  |4.0   |500    |[-1.1067320455782867,-0.44348778514743054,2.111937146904638]  |
|Laptop

In [13]:
# ----------------------------------------------------------------------------
# TUGAS 3: FEATURE ENGINEERING
# ----------------------------------------------------------------------------
print("\n" + "=" * 70)
print("TUGAS 3: FEATURE ENGINEERING")
print("=" * 70)

# 3a. Ekstrak bulan_rilis dari tgl_rilis
df_tugas_fe = df_tugas_scaled.withColumn(
    "timestamp_rilis",
    to_timestamp("tgl_rilis", "yyyy-MM-dd")
)
df_tugas_fe = df_tugas_fe.withColumn("bulan_rilis", month("timestamp_rilis"))

print("\n✅ Ekstraksi Bulan Rilis:")
df_tugas_fe.select("nama_produk", "tgl_rilis", "bulan_rilis").show()

# 3b. One-Hot Encoding untuk kategori dan status_stok
# Step 1: StringIndexer (ubah string → angka)
indexer_kategori = StringIndexer(inputCol="kategori", outputCol="kategori_index")
indexer_status = StringIndexer(inputCol="status_stok", outputCol="status_index")

df_tugas_indexed = indexer_kategori.fit(df_tugas_fe).transform(df_tugas_fe)
df_tugas_indexed = indexer_status.fit(df_tugas_indexed).transform(df_tugas_indexed)

# Step 2: OneHotEncoder (ubah angka → vektor binary)
ohe_tugas = OneHotEncoder(
    inputCols=["kategori_index", "status_index"],
    outputCols=["kategori_ohe", "status_ohe"]
)
df_tugas_encoded = ohe_tugas.fit(df_tugas_indexed).transform(df_tugas_indexed)

print("\n✅ One-Hot Encoding:")
df_tugas_encoded.select("kategori", "kategori_index", "kategori_ohe",
                        "status_stok", "status_index", "status_ohe").show(truncate=False)

# PENJELASAN ONE-HOT ENCODING:
# Contoh: kategori = ["Elektronik", "Aksesoris"]
# Elektronik → [1, 0]
# Aksesoris  → [0, 1]
# Ini supaya ML model bisa "baca" kategori


TUGAS 3: FEATURE ENGINEERING

✅ Ekstraksi Bulan Rilis:
+------------+----------+-----------+
| nama_produk| tgl_rilis|bulan_rilis|
+------------+----------+-----------+
| Headphone C|2023-02-15|          2|
|    Laptop A|2023-01-20|          1|
|Smartphone B|2023-02-10|          2|
|Smartwatch F|2023-04-12|          4|
|    Tablet D|2023-03-01|          3|
|   Charger E|2023-03-05|          3|
|    Laptop A|2023-01-20|          1|
+------------+----------+-----------+


✅ One-Hot Encoding:
+----------+--------------+-------------+-------------+------------+-------------+
|kategori  |kategori_index|kategori_ohe |status_stok  |status_index|status_ohe   |
+----------+--------------+-------------+-------------+------------+-------------+
|Aksesoris |1.0           |(1,[],[])    |stok_habis   |1.0         |(1,[],[])    |
|Elektronik|0.0           |(1,[0],[1.0])|stok_tersedia|0.0         |(1,[0],[1.0])|
|Elektronik|0.0           |(1,[0],[1.0])|stok_tersedia|0.0         |(1,[0],[1.0])|
|Elekt

In [15]:
# ----------------------------------------------------------------------------
# TUGAS 4:  HASIL AKHIR
# ----------------------------------------------------------------------------
print("\n" + "=" * 70)
print("TUGAS 4: HASIL AKHIR - Data Siap untuk ML")
print("=" * 70)

# Pilih kolom penting untuk ditampilkan
kolom_final = [
    "id_produk", "nama_produk", "kategori", "harga", "rating", "terjual",
    "bulan_rilis", "fitur_standar", "kategori_ohe", "status_ohe"
]

print("\n🎉 DataFrame Akhir (Bersih + Feature Engineering):")
df_tugas_encoded.select(kolom_final).show(10, truncate=False)

print("\n📊 Schema Akhir:")
df_tugas_encoded.select(kolom_final).printSchema()

# Statistik Deskriptif
print("\n📈 Statistik Deskriptif:")
df_tugas_encoded.select("harga", "rating", "terjual").describe().show()


TUGAS 4: HASIL AKHIR - Data Siap untuk ML

🎉 DataFrame Akhir (Bersih + Feature Engineering):
+---------+------------+----------+--------+------+-------+-----------+--------------------------------------------------------------+-------------+-------------+
|id_produk|nama_produk |kategori  |harga   |rating|terjual|bulan_rilis|fitur_standar                                                 |kategori_ohe |status_ohe   |
+---------+------------+----------+--------+------+-------+-----------+--------------------------------------------------------------+-------------+-------------+
|103      |Headphone C |Aksesoris |1200000 |4.2   |120    |2          |[-0.9511344112633691,-0.09855311527358197,-0.4888743395612589]|(1,[],[])    |(1,[],[])    |
|101      |Laptop A    |Elektronik|15000000|4.5   |120    |1          |[1.3091259608901722,0.4188497119263139,-0.4888743395612589]   |(1,[0],[1.0])|(1,[0],[1.0])|
|102      |Smartphone B|Elektronik|8000000 |4.7   |250    |2          |[0.16261707646446283

In [ ]:
# ----------------------------------------------------------------------------
# SELESAI - Stop Spark
# ----------------------------------------------------------------------------
print("\n" + "=" * 70)
print("✅ SEMUA LATIHAN DAN TUGAS SELESAI!")
print("=" * 70)
print("\n💡 Ringkasan yang sudah dikerjakan:")
print("   1. ✅ Setup PySpark")
print("   2. ✅ Latihan 1: Agregasi lanjutan")
print("   3. ✅ Latihan 2: Diskretisasi gaji")
print("   4. ✅ Latihan 3: Feature engineering (interaksi)")
print("   5. ✅ Tugas: Data Cleaning lengkap")
print("   6. ✅ Tugas: Data Transformation (standarisasi)")
print("   7. ✅ Tugas: Feature Engineering (ekstraksi + encoding)")
print("   8. ✅ Tugas: Hasil final siap untuk ML")
print("\n🚀 Data sudah siap digunakan untuk Machine Learning!")

# Stop SparkSession
spark.stop()
print("\n⏹️  SparkSession telah dihentikan.")


✅ SEMUA LATIHAN DAN TUGAS SELESAI!

💡 Ringkasan yang sudah dikerjakan:
   1. ✅ Setup PySpark
   2. ✅ Latihan 1: Agregasi lanjutan
   3. ✅ Latihan 2: Diskretisasi gaji
   4. ✅ Latihan 3: Feature engineering (interaksi)
   5. ✅ Tugas: Data Cleaning lengkap
   6. ✅ Tugas: Data Transformation (standarisasi)
   7. ✅ Tugas: Feature Engineering (ekstraksi + encoding)
   8. ✅ Tugas: Hasil final siap untuk ML

🚀 Data sudah siap digunakan untuk Machine Learning!

⏹️  SparkSession telah dihentikan.
